In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)
library(dplyr)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/cellranger_arc/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/genes/genes.gtf.gz"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
TSS_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.bed"
path.atac_frag = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/WTC11_HPC_lenient/cluster_5/atac_fragments.tsv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/WTC11_HPC_lenient/cluster_5/rna_count_matrix.csv.gz"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/WTC11_cluster5/WTC11_cluster5/Kendall/Pairs.tsv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/ArchR_250325/ArchR_250325/"
celltype = "WTC11_cluster5"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import candidate E-G pairs

In [5]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
# matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

make TSS GRanges

In [9]:
df.tss <- fread(TSS_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type"))
df.tss = as.data.frame(df.tss)
rownames(df.tss) = df.tss$name

In [10]:
df.tss[df.tss$strand == "+","end"] = df.tss[df.tss$strand == "+","start"]
df.tss[df.tss$strand == "-","start"] = df.tss[df.tss$strand == "-","end"]

In [11]:
df.tss = df.tss[unname(rownames(matrix.rna.rename)),]

In [12]:
gr.tss <- makeGRangesFromDataFrame(
  df.tss,
  seqnames.field = "chr",
  start.field = "start",
  end.field = "end",
  strand.field = "strand",    
  keep.extra.columns = TRUE   
)

In [13]:
gr.tss

GRanges object with 17524 ranges and 4 metadata columns:
            seqnames    ranges strand |        name     score      Ensembl_ID
               <Rle> <IRanges>  <Rle> | <character> <integer>     <character>
  LINC01128     chr1    827590      + |   LINC01128         0 ENSG00000228794
  LINC00115     chr1    827522      - |   LINC00115         0 ENSG00000225880
     FAM41C     chr1    876802      - |      FAM41C         0 ENSG00000230368
     SAMD11     chr1    925740      + |      SAMD11         0 ENSG00000187634
      NOC2L     chr1    959299      - |       NOC2L         0 ENSG00000188976
        ...      ...       ...    ... .         ...       ...             ...
     NLGN4Y     chrY  14523745      + |      NLGN4Y         0 ENSG00000165246
     TTTY14     chrY  19077547      - |      TTTY14         0 ENSG00000176728
      KDM5D     chrY  19744939      - |       KDM5D         0 ENSG00000012817
     TTTY10     chrY  20519228      - |      TTTY10         0 ENSG00000229236
     EI

Create an ArchRProject

In [14]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(path.atac_frag),
  sampleNames = c(celltype),
  validBarcodes = list(WTC11_cluster5 = colnames(matrix.rna.rename)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-2b40a56cf3220f-Date-2025-03-25_Time-16-39-45.884567.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2025-03-25 16:39:46.30563 : Batch Execution w/ safelapply!, 0 mins elapsed.

(WTC11_cluster5 : 1 of 1) Determining Arrow Method to use!

2025-03-25 16:39:46.381244 : (WTC11_cluster5 : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.001 mins elapsed.

2025-03-25 16:39:46.392265 : (WTC11_cluster5 : 1 of 1) Tabix Bed To Temporary File, 0.001 mins elapsed.

2025-03-25 16:40:51.968513 : (WTC11_cluster5 : 1 of 1) Successful creation of Temporary File, 1.094 mins elapsed.

2025-03-25 16:40:51.976239 : (WTC11_cluster5 : 1 of 1) Creating ArrowFile From Temporary File, 1.095 mins elapsed.

2025-03-25 16:41:17.049677 : (WTC11_cluster5 : 1 of 1) Successful creation of Arrow File, 1.512 min

In [15]:
ArrowFiles

[1] "WTC11_cluster5.arrow"

In [16]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

In [17]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [18]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/ArchR_250325/ArchR_250325/ArchR 
samples(1): WTC11_cluster5
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 791
medianTSS(1): 11.204
medianFrags(1): 53648

In [19]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-2b40a55e5c876b-Date-2025-03-25_Time-16-48-19.663651.log
If there is an issue, please report to github with logFile!

2025-03-25 16:48:20.293555 : Computing Total Across All Features, 0.001 mins elapsed.

2025-03-25 16:48:22.5281 : Computing Top Features, 0.038 mins elapsed.

###########
2025-03-25 16:48:25.532104 : Running LSI (1 of 2) on Top Features, 0.088 mins elapsed.
###########

2025-03-25 16:48:25.552024 : Creating Partial Matrix, 0.088 mins elapsed.

2025-03-25 16:48:41.135012 : Computing LSI, 0.348 mins elapsed.

2025-03-25 16:49:27.774069 : Identifying Clusters, 1.125 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2025-03-25 16:49:32.817006 : Identified 6 Clusters, 1.209 mins elapsed.

2025-03-25 16:49:32.837105 : Saving LSI Iteration, 1.21 mins elapsed.

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

Fou


************************************************************
2025-03-25 16:49:45.519584 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-2b40a55e5c876b-Date-2025-03-25_Time-16-48-19.663651.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2025-03-25 16:49:45.523005 : Creating Cluster Matrix on the total Group Features, 1.421 mins elapsed.

2025-03-25 16:50:02.598005 : Computing Variable Features, 1.706 mins elapsed.

###########
2025-03-25 16:50:02.778947 : Running LSI (2 of 2) on Variable Features, 1.709 mins elapsed.
###########

2025-03-25 16:50:02.79814 : Creating Partial Matrix, 1.709 mins elapsed.

2025-03-25 16:50:12.254971 : Computing LSI, 1.867 mins elapsed.

2025-03-25 16:50:59.329499 : Finished Running IterativeLSI, 2.651 mins elapsed.



In [20]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [21]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-2b40a5a79c8e5-Date-2025-03-25_Time-16-51-03.178492.log
If there is an issue, please report to github with logFile!

2025-03-25 16:51:03.458099 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-25 16:51:04.209887 : Adding WTC11_cluster5 to PeakMatrix for Chr (1 of 23)!, 0.011 mins elapsed.

2025-03-25 16:51:13.771046 : Adding WTC11_cluster5 to PeakMatrix for Chr (2 of 23)!, 0.171 mins elapsed.

2025-03-25 16:51:21.871066 : Adding WTC11_cluster5 to PeakMatrix for Chr (3 of 23)!, 0.306 mins elapsed.

2025-03-25 16:51:29.052075 : Adding WTC11_cluster5 to PeakMatrix for Chr (4 of 23)!, 0.426 mins elapsed.

2025-03-25 16:51:35.899617 : Adding WTC11_cluster5 to PeakMatrix for Chr (5 of 23)!, 0.54 mins elapsed.

2025-03-25 16:51:42.286648 : Adding WTC11_cluster5 to PeakMatrix for Chr (6 of 23)!, 0.646 mins elapsed.

2025-03-25 16:51:48.759189 : Adding WTC11_cluster5 to PeakMatrix for Chr (7 of 23)!, 0.754 mins elapsed.

2025-03-25 16:51

In [22]:
matrix.rna.rename2 = matrix.rna.rename
colnames(matrix.rna.rename2) = paste(celltype,colnames(matrix.rna.rename),sep="#")

In [23]:
seRNA <- SummarizedExperiment(
  assays = SimpleList(counts = matrix.rna.rename2),
  rowRanges = gr.tss
)

In [24]:
seRNA = sort.GenomicRanges(sortSeqlevels(seRNA), 
                           ignore.strand = TRUE)

In [25]:
seRNA

class: RangedSummarizedExperiment 
dim: 17524 791 
metadata(0):
assays(1): counts
rownames(17524): LINC00115 LINC01128 ... TTTY10 EIF1AY
rowData names(4): name score Ensembl_ID gene_type
colnames(791): WTC11_cluster5#AAACAGCCAAACCCTA-1
  WTC11_cluster5#AAACCAACAACATAAG-1 ...
  WTC11_cluster5#TTTGTCTAGATAACCC-1 WTC11_cluster5#TTTGTGGCAGGCAAGC-1
colData names(0):

In [26]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-2b40a52ba866f3-Date-2025-03-25_Time-16-53-48.961231.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2025-03-25 16:53:49.45887 : 

Overlap Per Sample w/ scATAC : WTC11_cluster5=791

2025-03-25 16:53:49.46997 : 

2025-03-25 16:53:50.984147 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-25 16:53:51.612918 : Adding WTC11_cluster5 to GeneExpressionMatrix for Chr (1 of 23)!, 0.01 mins elapsed.

2025-03-25 16:53:58.516363 : Adding WTC11_cluster5 to GeneExpressionMatrix for Chr (2 of 23)!, 0.126 mins elapsed.

2025-03-25 16:54:05.131032 : Adding WTC11_cluster5 to GeneExpressionMatrix for Chr (3 of 23)!, 0.236 mins elapsed.

2025-03-25 16:54:14.47154 : Adding WTC11_cluster5 to GeneExpressionMatrix for Chr (4 of 23)!, 0.391 mins elapsed.

2025-03-25 16:54:20.630713 : Adding WTC11_cluster5 to GeneExpressionMatrix for Chr (5 of 23)!, 0.494 mins elapsed.

2025-03-25 16:54:26.26633 : Adding

Run ArchR prediction

In [27]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-2b40a56b06f58f-Date-2025-03-25_Time-16-56-36.17321.log
If there is an issue, please report to github with logFile!

2025-03-25 16:56:36.605271 : Getting Available Matrices, 0.007 mins elapsed.

No predictionScore found. Continuing without predictionScore!

2025-03-25 16:56:36.920839 : Filtered Low Prediction Score Cells (0 of 791, 0), 0 mins elapsed.

2025-03-25 16:56:37.031521 : Computing KNN, 0.002 mins elapsed.

2025-03-25 16:56:37.073392 : Identifying Non-Overlapping KNN pairs, 0.003 mins elapsed.

2025-03-25 16:56:38.597563 : Identified 489 Groupings!, 0.028 mins elapsed.

2025-03-25 16:56:38.635398 : Getting Group RNA Matrix, 0.029 mins elapsed.

2025-03-25 16:56:58.645332 : Getting Group ATAC Matrix, 0.362 mins elapsed.

2025-03-25 16:57:54.148563 : Normalizing Group Matrices, 1.287 mins elapsed.

2025-03-25 16:58:03.018608 : Finding Peak Gene Pairings, 1.435 mins elapsed.

2025-03-25 16:58:04.2269 : Computing Correlations, 1.

In [28]:
execution_time

Time difference of 1.781427 mins

In [29]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [30]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [31]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [32]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 2657764 rows and 7 columns
                                idxATAC    idxRNA Correlation         FDR
                              <integer> <integer>   <numeric>   <numeric>
chr1-10021-10505_LINC00115            1         1  -0.1141958  0.03608648
chr1-180699-181904_LINC00115          2         1   0.1644095  0.00169764
chr1-191227-191617_LINC00115          3         1   0.0509998  0.39804127
chr1-267886-268117_LINC00115          4         1   0.0578537  0.32961268
chr1-778320-779311_LINC00115          5         1  -0.1181020  0.02954749
...                                 ...       ...         ...         ...
chrX-155966905-155967206_IL9R    152400     17509  -0.0042053 9.54278e-01
chrX-155971502-155972157_IL9R    152401     17509   0.0704830 2.22976e-01
chrX-156000454-156000883_IL9R    152402     17509  -0.0390446 5.31823e-01
chrX-156029831-156030296_IL9R    152403     17509  -0.0449059 4.64193e-01
chrX-156030503-156030719_IL9R    152404     17509   0.3442538 1.19022e

In [33]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [34]:
pairs.E2G.res

GRanges object with 2154019 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034573-100034728      * |         AGL
        [2]     chr1 100034573-100034728      * |      CDC14A
        [3]     chr1 100034573-100034728      * |         DBT
        [4]     chr1 100034573-100034728      * |        DPH5
        [5]     chr1 100034573-100034728      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2154015]     chrX     9995697-9996195      * |       CLCN4
  [2154016]     chrX     9995697-9996195      * |      GPR143
  [2154017]     chrX     9995697-9996195      * |     SHROOM2
  [2154018]     chrX     9995697-9996195      * |       TBL1X
  [2154019]     chrX     9995697-9996195      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

Save results

In [35]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 2154019 ranges and 7 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034573-100034728      * |         AGL
        [2]     chr1 100034573-100034728      * |      CDC14A
        [3]     chr1 100034573-100034728      * |         DBT
        [4]     chr1 100034573-100034728      * |        DPH5
        [5]     chr1 100034573-100034728      * |       EXTL2
        ...      ...                 ...    ... .         ...
  [2154015]     chrX     9995697-9996195      * |       CLCN4
  [2154016]     chrX     9995697-9996195      * |      GPR143
  [2154017]     chrX     9995697-9996195      * |     SHROOM2
  [2154018]     chrX     9995697-9996195      * |       TBL1X
  [2154019]     chrX     9995697-9996195      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <nume

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100034573,100034728,AGL,WTC11_cluster5,-0.0436631367,4.782771e-01,0.04240046,0.8243760
chr1,100034573,100034728,CDC14A,WTC11_cluster5,0.1292985683,1.610183e-02,0.04240046,0.8577874
chr1,100034573,100034728,DBT,WTC11_cluster5,-0.0286173399,6.586870e-01,0.04240046,0.6375578
chr1,100034573,100034728,DPH5,WTC11_cluster5,0.0842369437,1.361776e-01,0.04240046,0.7315666
chr1,100034573,100034728,EXTL2,WTC11_cluster5,0.1165767724,3.197162e-02,0.04240046,0.8620709
chr1,100034573,100034728,MFSD14A,WTC11_cluster5,-0.0036682992,9.602563e-01,0.04240046,0.7301388
chr1,100034573,100034728,PLPPR4,WTC11_cluster5,0.0140590809,8.389499e-01,0.04240046,0.2813981
chr1,100034573,100034728,RTCA,WTC11_cluster5,-0.1676390107,1.344091e-03,0.04240046,0.7156891
chr1,100034573,100034728,RTCA-AS1,WTC11_cluster5,-0.0000366005,9.996019e-01,0.04240046,0.2836256


In [36]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] uwot_0.1.16           